In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import subprocess
import os
from six.moves import urllib
import tempfile
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_non_alphanum
from unidecode import unidecode
import spacy
import re
import xml
spacy.prefer_gpu()
# sp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
TAG_RE = re.compile(r'<[^>]+>')
import util as ut
data_filename = '../beauty.csv'

TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

In [64]:
def strip_custom(token):
    token = token.replace('&Reg;', " ")
    token = token.replace("&lt;", "<")
    token = token.replace("&times;", "")
    token = token.replace("&gt;", ">")
    token = token.replace("&quot;", "")
    token = token.replace('&nbsp', " ")
    token = token.replace('&copy;', " ")
    token = token.replace('&reg', " ")
    token = token.replace('%20', " ")
    # this has to be last:
    token = token.replace("&amp;", "&")
    token = token.replace("â\x80¢", " ")
    token = token.replace("Â®", " ")
    token = token.replace("Ã©", " ")
    token = token.replace("®", " ")
    token = token.replace("©", " ")
    token = token.replace("™", " ")
    token = token.replace("•", "")
    token = token.replace("�", "")
    
    token = token.replace("width:99pt", "")
    token = token.replace('class="xl66">', '')
    #token = re.sub(r"\'", '', token)
    token = token.replace('&#160;', ' ')
    return token


def string_processor(token):

#     str = unidecode(token)
    str = token.lower()
    str = strip_custom(str)
    str = remove_stopwords(str)
    str = strip_punctuation(str)
    str = remove_tags(str)
    str = strip_non_alphanum(str)
#     tokens = sp(str)
#     tokens = [token.lemma_ for token in tokens]
#     str = " ".join(tokens)
    str = strip_multiple_whitespaces(str)

    return str

In [5]:
def processing_sequences(x_test,maxlen=16,ngram_range=1):
    print(len(x_test), 'test sequences')

    print('Average test sequence length: {}'.format(
        np.mean(list(map(len, x_test)))))

    if ngram_range > 1:
        print('Adding {}-gram features'.format(ngram_range))
        # Create set of unique n-gram from the training set.
        ngram_set = set()
        for input_list in x_train:
            for i in range(2, ngram_range + 1):
                set_of_ngram = create_ngram_set(input_list, ngram_value=i)
                ngram_set.update(set_of_ngram)

        # Dictionary mapping n-gram token to a unique integer.
        # Integer values are greater than max_features in order
        # to avoid collision with existing features.
        start_index = max_features + 1
        token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
        indice_token = {token_indice[k]: k for k in token_indice}

        # max_features is the highest integer that could be found in the dataset.
        max_features = np.max(list(indice_token.keys())) + 1
        print("max_features:",max_features)

        # Augmenting x_train and x_test with n-grams features

        x_test = add_ngram(x_test, token_indice, ngram_range)

        print('Average test sequence length: {}'.format(
            np.mean(list(map(len, x_test)), dtype=int)))

    print('Pad sequences (samples x time)')

    x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

    print('x_test shape:', x_test.shape)
    return x_test

In [6]:
df = pd.read_csv(data_filename)
df = df[['bucket_name','product_name']]
print("before drop: ",len(df))
df = df.dropna()
print("after drop: ",len(df))
# to lowercase
df.product_name = df.product_name.apply(lambda x : str(x).lower())
df.product_name = df.product_name.apply(lambda x : string_processor(x))

corpus = df.product_name.tolist()


vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size, 
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
                      lower=True, 
                      split=' ')

tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
sequences = np.array(sequences)


X_test = processing_sequences(sequences,maxlen=16,ngram_range=1)
# X = np.array(sequences)
# y = np.array(label)

before drop:  87877
after drop:  87854
87854 test sequences
Average test sequence length: 6.957190338516175
Pad sequences (samples x time)
x_test shape: (87854, 16)


In [7]:
import googleapiclient.discovery

def predict_json(project, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']


# def get_true_label(encoder,y_hats):
#     labels = [np.argmax(np.array(i['dense'])) for i in y_hats]
#     return encoder.inverse_transform(labels),labels
    

ImportError: No module named 'googleapiclient'

In [19]:
inc = X_test[33137:33137+30].tolist()
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.classes_ = np.load('saved_encoder.npy',allow_pickle=True)

In [20]:
y_hat = predict_json('groupby-development','beautyPredictModel',inc)

In [21]:
def getTopK(array,K):
    ind = np.argpartition(array, -K)[-K:][::-1]
    prob = array[ind]
    return ind, prob

def topK_DF(y_hat,K):
    col = []
    for i in range(1,K+1):
        col.append('Prediction'+str(i))
        col.append('Probs'+str(i))

    df_list = []
    for i in y_hat:
        labels,prob = getTopK(np.array(i['dense']),K)
        classes = encoder.inverse_transform(labels)

        result_list = []
        for c,p in zip(classes,prob):
            result_list.append(c)
            result_list.append(p)
        df_list.append(result_list)
    
    return pd.DataFrame(df_list,columns=col)


In [22]:
predict_df = topK_DF(y_hat,3)
predict_df

,Prediction1,Probs1,Prediction2,Probs2,Prediction3,Probs3
0,lotions & moisturizers,0.330705,hand sanitizers,0.196707,"pill cases, organizers & accessories",0.121946
1,lotions & moisturizers,0.986280,serums & treatments,0.002268,hand sanitizers,0.002004
2,lotions & moisturizers,0.986280,serums & treatments,0.002268,hand sanitizers,0.002004
3,lotions & moisturizers,0.920290,hand sanitizers,0.017991,hand soaps,0.017394
4,lotions & moisturizers,0.987075,hand sanitizers,0.003121,serums & treatments,0.002967
5,lotions & moisturizers,0.987075,hand sanitizers,0.003121,serums & treatments,0.002967
6,lotions & moisturizers,0.751206,serums & treatments,0.209895,skin care variety packs,0.008195
7,lotions & moisturizers,0.154417,serums & treatments,0.788894,facial cleansers & cosmetic removers,0.034634
8,lotions & moisturizers,0.803832,skin treatments,0.077266,sun care & tanning,0.038654
9,lotions & moisturizers,0.985399,face & body oils,0.004613,skin treatments,0.002767
